In [340]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
from sklearn import svm
import warnings
warnings.filterwarnings("ignore")


import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

### Combine multiple CSV's

In [341]:
import os

folder_path = r'C:\Users\shrey\Downloads\Signature-Forgery-Detection-main\Features\Training'

all_files = os.listdir(folder_path)

# Filter out non-CSV files
csv_files = [f for f in all_files if f.endswith('.csv')]

# Create a list to hold the dataframes
df_list = []

for csv in csv_files:
    file_path = os.path.join(folder_path, csv)
    try:
        # Try reading the file using default UTF-8 encoding
        df = pd.read_csv(file_path)
        df_list.append(df)
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try reading the file using UTF-16 encoding with tab separator
            df = pd.read_csv(file_path, sep='\t', encoding='utf-16')
            df_list.append(df)
        except Exception as e:
            print(f"Could not read file {csv} because of error: {e}")
    except Exception as e:
        print(f"Could not read file {csv} because of error: {e}")

# Concatenate all data into one DataFrame
big_df = pd.concat(df_list, ignore_index=True)

# Save the final result to a new CSV file
big_df.to_csv(os.path.join(folder_path, 'combined_train_file.csv'), index=False)

### Analzying Training Set

In [342]:
train_df = pd.read_csv(r'C:\Users\shrey\Downloads\Signature-Forgery-Detection-main\Features\Training\combined_train_file.csv')
train_df.shape

(936, 10)

In [343]:
train_df.head(2)

,ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output
0,0.107930,0.484153,0.476339,0.977854,0.121258,0.136598,-0.135199,-1.535753,0.148335,1
1,0.102177,0.485792,0.467730,0.966284,0.120302,0.242219,-0.188648,-1.518732,-0.136041,1


In [344]:
train_df['output'].value_counts()

output
1    468
0    468
Name: count, dtype: int64

In [345]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 936 entries, 0 to 935
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ratio         936 non-null    float64
 1   cent_y        936 non-null    float64
 2   cent_x        936 non-null    float64
 3   eccentricity  936 non-null    float64
 4   solidity      936 non-null    float64
 5   skew_x        936 non-null    float64
 6   skew_y        936 non-null    float64
 7   kurt_x        936 non-null    float64
 8   kurt_y        936 non-null    float64
 9   output        936 non-null    int64  
dtypes: float64(9), int64(1)
memory usage: 73.2 KB


In [346]:
corr = train_df.corr()
corr.style.background_gradient(cmap='coolwarm')

,ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output
ratio,1.000000,0.147584,-0.224684,0.114385,0.846605,0.023879,-0.027701,0.028645,-0.274704,0.043942
cent_y,0.147584,1.000000,0.076727,0.325886,-0.027747,-0.102087,-0.841610,-0.214716,0.483154,-0.192973
cent_x,-0.224684,0.076727,1.000000,0.307239,-0.382510,-0.692244,-0.041053,-0.432422,0.301114,-0.170029
eccentricity,0.114385,0.325886,0.307239,1.000000,0.121168,-0.586018,-0.325391,-0.594864,0.150248,0.154636
solidity,0.846605,-0.027747,-0.382510,0.121168,1.000000,0.154593,0.151571,0.259625,-0.378683,0.133537
skew_x,0.023879,-0.102087,-0.692244,-0.586018,0.154593,1.000000,0.070625,0.675592,-0.024773,0.189134
skew_y,-0.027701,-0.841610,-0.041053,-0.325391,0.151571,0.070625,1.000000,0.275622,-0.472338,0.133819
kurt_x,0.028645,-0.214716,-0.432422,-0.594864,0.259625,0.675592,0.275622,1.000000,0.005965,0.054378
kurt_y,-0.274704,0.483154,0.301114,0.150248,-0.378683,-0.024773,-0.472338,0.005965,1.000000,0.023669
output,0.043942,-0.192973,-0.170029,0.154636,0.133537,0.189134,0.133819,0.054378,0.023669,1.000000


### Analyzing Testing Set 

In [347]:
import os

folder_path = r'C:\Users\shrey\Downloads\Signature-Forgery-Detection-main\Features\Testing'

all_files = os.listdir(folder_path)

# Filter out non-CSV files
csv_files = [f for f in all_files if f.endswith('.csv')]

# Create a list to hold the dataframes
df_list = []

for csv in csv_files:
    file_path = os.path.join(folder_path, csv)
    try:
        # Try reading the file using default UTF-8 encoding
        df = pd.read_csv(file_path)
        df_list.append(df)
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try reading the file using UTF-16 encoding with tab separator
            df = pd.read_csv(file_path, sep='\t', encoding='utf-16')
            df_list.append(df)
        except Exception as e:
            print(f"Could not read file {csv} because of error: {e}")
    except Exception as e:
        print(f"Could not read file {csv} because of error: {e}")

# Concatenate all data into one DataFrame
big_df = pd.concat(df_list, ignore_index=True)

# Save the final result to a new CSV file
big_df.to_csv(os.path.join(folder_path, 'combined_test_file.csv'), index=False)

In [348]:
test_df = pd.read_csv(r'C:\Users\shrey\Downloads\Signature-Forgery-Detection-main\Features\Testing\combined_test_file.csv')
test_df.shape

(576, 10)

In [349]:
test_df.head(2)

,ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output
0,0.106640,0.475430,0.444786,0.968660,0.126933,0.308466,-0.177807,-1.483313,-0.028273,1
1,0.101542,0.481809,0.493841,0.974536,0.114908,0.108399,-0.136275,-1.569744,-0.090782,1


In [350]:
test_df.output.value_counts()

output
1    288
0    288
Name: count, dtype: int64

In [351]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ratio         576 non-null    float64
 1   cent_y        576 non-null    float64
 2   cent_x        576 non-null    float64
 3   eccentricity  576 non-null    float64
 4   solidity      576 non-null    float64
 5   skew_x        576 non-null    float64
 6   skew_y        576 non-null    float64
 7   kurt_x        576 non-null    float64
 8   kurt_y        576 non-null    float64
 9   output        576 non-null    int64  
dtypes: float64(9), int64(1)
memory usage: 45.1 KB


### Model Training

In [352]:
train_df.drop(['ratio'], axis=1, inplace=True)

In [353]:
test_df.drop(['ratio'], axis=1, inplace=True)

In [354]:
train_df.drop(['eccentricity'], axis=1, inplace=True)

In [355]:
test_df.drop(['eccentricity'], axis=1, inplace=True)

In [356]:
train_df.head(2)

,cent_y,cent_x,solidity,skew_x,skew_y,kurt_x,kurt_y,output
0,0.484153,0.476339,0.121258,0.136598,-0.135199,-1.535753,0.148335,1
1,0.485792,0.467730,0.120302,0.242219,-0.188648,-1.518732,-0.136041,1


In [357]:
corr = train_df.corr()
corr.style.background_gradient(cmap='coolwarm')

,cent_y,cent_x,solidity,skew_x,skew_y,kurt_x,kurt_y,output
cent_y,1.000000,0.076727,-0.027747,-0.102087,-0.841610,-0.214716,0.483154,-0.192973
cent_x,0.076727,1.000000,-0.382510,-0.692244,-0.041053,-0.432422,0.301114,-0.170029
solidity,-0.027747,-0.382510,1.000000,0.154593,0.151571,0.259625,-0.378683,0.133537
skew_x,-0.102087,-0.692244,0.154593,1.000000,0.070625,0.675592,-0.024773,0.189134
skew_y,-0.841610,-0.041053,0.151571,0.070625,1.000000,0.275622,-0.472338,0.133819
kurt_x,-0.214716,-0.432422,0.259625,0.675592,0.275622,1.000000,0.005965,0.054378
kurt_y,0.483154,0.301114,-0.378683,-0.024773,-0.472338,0.005965,1.000000,0.023669
output,-0.192973,-0.170029,0.133537,0.189134,0.133819,0.054378,0.023669,1.000000


### training set

In [358]:
x_train = train_df.drop(['output'],axis=1)
y_train = train_df['output']

### testing set

In [359]:
x_test = test_df.drop(['output'], axis=1)
y_test = test_df['output']

In [360]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),    # Standardize features
    ('pca', PCA(n_components=0.95)), # Reduce dimensionality
    ('svm', SVC())                   # SVM model
])

In [361]:
param_grid = {
    'svm__C': [0.1, 1, 10, 100],
    'svm__gamma': [1, 0.1, 0.01, 0.001],
    'svm__kernel': ['rbf', 'poly', 'sigmoid']
}

In [362]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA(n_components=0.95)),
                                       ('svm', SVC())]),
             n_jobs=-1,
             param_grid={'svm__C': [0.1, 1, 10, 100],
                         'svm__gamma': [1, 0.1, 0.01, 0.001],
                         'svm__kernel': ['rbf', 'poly', 'sigmoid']},
             scoring='accuracy', verbose=2)

In [363]:
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation Accuracy: {grid_search.best_score_}")

Best Parameters: {'svm__C': 1, 'svm__gamma': 1, 'svm__kernel': 'rbf'}
Best Cross-Validation Accuracy: 1.0


In [364]:
best_model = grid_search.best_estimator_
test_accuracy = best_model.score(x_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

Test Accuracy: 0.7291666666666666


### Scaling

In [365]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train),columns = x_train.columns) 
x_train.head()

,cent_y,cent_x,solidity,skew_x,skew_y,kurt_x,kurt_y
0,-0.509245,0.309395,-1.146946,0.029665,0.152384,-1.281788,0.431331
1,-0.484859,0.108271,-1.159866,0.521097,0.041107,-1.239240,0.029655
2,-0.452236,0.142027,-1.308280,0.279744,-0.367687,-1.294730,0.314063
3,0.777305,0.649976,-1.119974,-0.207948,-1.567031,-0.989261,0.772181
4,1.334659,0.304146,-0.740590,0.065596,-1.546631,-0.990195,0.601191


In [366]:
x_test = pd.DataFrame(scaler.fit_transform(x_test),columns = x_test.columns) 
x_test.head()

,cent_y,cent_x,solidity,skew_x,skew_y,kurt_x,kurt_y
0,-0.776097,-0.208269,-1.104603,0.551038,0.062367,-1.140744,0.138569
1,-0.685720,0.886258,-1.282639,-0.240007,0.143103,-1.350952,0.070971
2,0.965354,-0.155908,-0.873440,0.091166,-1.074067,-0.990652,0.127274
3,0.245575,-0.180823,-1.129008,0.185822,-1.263002,-1.151396,0.106536
4,-1.640588,0.090338,-0.500503,-0.952855,0.875174,-0.159174,-0.747399


In [367]:
from sklearn.svm import SVC
svc = SVC(kernel='rbf',C=1.2)
svc.fit(x_train, y_train)

SVC(C=1.2)

In [379]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(svc,scaler,target,cv=5)
print(score.mean())

NameError: name 'target' is not defined

In [368]:
y_pred = svc.predict(x_test)

In [369]:
model_pred = pd.DataFrame(y_pred)


In [370]:
test_df.reset_index(drop=True, inplace=True)
model_pred.reset_index(drop=True, inplace=True)

In [371]:
model_pred.head()

,0
0,1
1,1
2,0
3,1
4,1


In [372]:
res_df = pd.concat([test_df,model_pred], axis=1, )
res_df.head()

,cent_y,cent_x,solidity,skew_x,skew_y,kurt_x,kurt_y,output,0
0,0.475430,0.444786,0.126933,0.308466,-0.177807,-1.483313,-0.028273,1,1
1,0.481809,0.493841,0.114908,0.108399,-0.136275,-1.569744,-0.090782,1,1
2,0.598349,0.447133,0.142548,0.192157,-0.762412,-1.421599,-0.038718,0,0
3,0.547544,0.446016,0.125285,0.216097,-0.859603,-1.487693,-0.057895,0,1
4,0.414410,0.458169,0.167738,-0.071892,0.240317,-1.079719,-0.847548,1,1


In [373]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,y_pred))
print(metrics.accuracy_score(y_test,y_pred))
print(metrics.classification_report(y_test,y_pred))

[[228  60]
 [ 36 252]]
0.8333333333333334
              precision    recall  f1-score   support

           0       0.86      0.79      0.83       288
           1       0.81      0.88      0.84       288

    accuracy                           0.83       576
   macro avg       0.84      0.83      0.83       576
weighted avg       0.84      0.83      0.83       576

